# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import nltk

import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import sqlalchemy
from sqlalchemy import create_engine

from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [5]:
conn.close()

In [7]:

# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
conn = engine.connect()

df = pd.read_sql_table('Messages1', conn)


In [8]:
df 

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26023,30261,The training demonstrated how to enhance micro...,news,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26024,30262,A suitable candidate has been selected and OCH...,news,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26025,30263,"Proshika, operating in Cox's Bazar municipalit...",news,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26026,30264,"Some 2,000 women protesting against the conduc...",news,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#create Dependant(y) and independant(X) variables
X = df['message']
y = df.drop(columns =['id','message','genre'])
  

In [11]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26023    The training demonstrated how to enhance micro...
26024    A suitable candidate has been selected and OCH...
26025    Proshika, operating in Cox's Bazar municipalit...
26026    Some 2,000 women protesting against the conduc...
26027    A radical shift in thinking came about as a re...
Name: message, Length: 26028, dtype: object

In [12]:
df.head()

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    '''
    The function breaks a text into smaller components, typically words, phrases, symbols,
    or other meaningful elements, which are called tokens
    
    :param text: text to be tokenized
    :return :tokenized text
    '''
    # mormilize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    words = word_tokenize(text)
    
    #remove stop words
    #words = [lemmatizer.lemmatize(word) 
    words = [w for w in words if w not in stopwords.words("english")]
    
     #reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
# create an ML pipeline
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
#Spliting the data into the train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)



In [17]:
#fitting the data into the pipeline
pipeline.fit(X_train, y_train)

C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001C3170EFAC0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [19]:
# Printing the classification report for each label
def model_report(y_test, y_pred):
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [20]:
model_report(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.73      0.41      0.52      1838
           1       0.84      0.95      0.89      5971

    accuracy                           0.83      7809
   macro avg       0.78      0.68      0.71      7809
weighted avg       0.81      0.83      0.81      7809

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6503
           1       0.85      0.51      0.63      1306

    accuracy                           0.90      7809
   macro avg       0.88      0.74      0.79      7809
weighted avg       0.90      0.90      0.89      7809

Feature 3: offer


C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7779
           1       0.00      0.00      0.00        30

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.99      1.00      0.99      7809

Feature 4: aid_related
              precision    recall  f1-score   support

           0       0.80      0.85      0.82      4556
           1       0.77      0.70      0.74      3253

    accuracy                           0.79      7809
   macro avg       0.79      0.78      0.78      7809
weighted avg       0.79      0.79      0.79      7809

Feature 5: medical_help
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      7203
           1       0.68      0.08      0.14       606

    accuracy                           0.93      7809
   macro avg       0.80      0.54      0.55      7809
weighted avg       0.91    

C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7637
           1       0.79      0.06      0.12       172

    accuracy                           0.98      7809
   macro avg       0.88      0.53      0.55      7809
weighted avg       0.98      0.98      0.97      7809

Feature 35: other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      7368
           1       0.74      0.03      0.06       441

    accuracy                           0.94      7809
   macro avg       0.84      0.52      0.52      7809
weighted avg       0.93      0.94      0.92      7809

Feature 36: direct_report
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      6295
           1       0.81      0.37      0.51      1514

    accuracy                           0.86      7809
   macro avg       0.84      0.67      0.71      7809
weighted avg       0.8

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001C3170EFAC0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001C3170EFAC0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [53]:
# Define the parameter grid
paramGrid = {
#     'vect__ngram_range': [(1, 1), (1, 2)],
#     'vect__max_df': [0.5, 0.75, 1.0],
#     'vect__max_features': [None, 5000, 10000],
#     'tfidf__use_idf': [True, False],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__bootstrap': [True, False]}

In [54]:
# create grid search object 
cv = GridSearchCV(pipeline, param_grid=paramGrid,cv=3,n_jobs=1,scoring="accuracy")

cv

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001C3170EFAC0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=1,
             param_grid={'clf__estimator__bootstrap': [True, False],
                         'clf__estimator__n_estimators': [50, 100]},
             scoring='accuracy')

In [55]:
# trying all the parameter combinations and performing cross-validation
cv.fit(X_train, y_train)

C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_patt

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001C3170EFAC0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=1,
             param_grid={'clf__estimator__bootstrap': [True, False],
                         'clf__estimator__n_estimators': [50, 100]},
             scoring='accuracy')

In [56]:
cv.best_params_

{'clf__estimator__bootstrap': True, 'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [57]:
y_pred = cv.predict(X_test)

In [58]:
model_report(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.72      0.41      0.52      1838
           1       0.84      0.95      0.89      5971

    accuracy                           0.82      7809
   macro avg       0.78      0.68      0.71      7809
weighted avg       0.81      0.82      0.80      7809

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6503
           1       0.84      0.50      0.63      1306

    accuracy                           0.90      7809
   macro avg       0.87      0.74      0.79      7809
weighted avg       0.90      0.90      0.89      7809

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7779
           1       0.00      0.00      0.00        30

    accuracy                           1.00      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       

C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      7672
           1       0.33      0.01      0.01       137

    accuracy                           0.98      7809
   macro avg       0.66      0.50      0.50      7809
weighted avg       0.97      0.98      0.97      7809

Feature 9: military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7540
           1       0.67      0.04      0.08       269

    accuracy                           0.97      7809
   macro avg       0.82      0.52      0.53      7809
weighted avg       0.96      0.97      0.95      7809

Feature 10: child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7809

    accuracy                           1.00      7809
   macro avg       1.00      1.00      1.00      7809
weighted avg       1.00      1.00      1.00      7809

Feature 11: water
           

C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chabalaa\AppData\Local\anaconda3\conda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7460
           1       0.00      0.00      0.00       349

    accuracy                           0.95      7809
   macro avg       0.48      0.50      0.49      7809
weighted avg       0.91      0.95      0.93      7809

Feature 29: weather_related
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      5581
           1       0.85      0.71      0.78      2228

    accuracy                           0.88      7809
   macro avg       0.87      0.83      0.85      7809
weighted avg       0.88      0.88      0.88      7809

Feature 30: floods
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7148
           1       0.91      0.51      0.65       661

    accuracy                           0.95      7809
   macro avg       0.93      0.75      0.81      7809
weighted avg       0.95    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [59]:
import pickle
#Export the model to a pickle file.
pickle_file_path = "model.pkl"

with open(pickle_file_path, "wb") as pickle_file:
    pickle.dump(pipeline, pickle_file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.